# Studying COVID-19 Cases in New York

### 1. Introduction

Day by day COVID-19 cases are increasing and everyone is concern about going out. Nobody knows if they are virus carriers as sometimes they have a strong immunity system that is enough to keep that virus away or at least they don't feel sick. Are poplular shops the reason? Do they have to be locked until this pandemic is gone? From my own prespective, popular shops often have a high chance to have more cases than unpopular ones. The most common shops that most people visit are the ones which should be closed because most likely the most cases came from similar shops.

The audience I'm targeting can be the shop owners or anyone interested in knowing what shops have a high chance to be closed.

### 2. Methodology

1. Data analysis.
2. Foursquare.
3. Clustering.
4. Folium.

### 3. Results

![venues](https://imgur.com/rELQSAD.png)
![map](https://imgur.com/l15bf62.png)
![map2](https://imgur.com/gtjaiyr.png)

### 4. Conclusion

The main goal of this project was to target the popular / most common venues that have most likely high impact on COVID-19 cases thus we first cleaned our dataset to have everything ready for analysis then we managed to target Queens borough due to being afraid to risk Foursquare rate limits. Next we set up Foursquare to help us gather the venues by exploring around the coordiantes of each neighborhood resulting in extracting a total of 10112 venues. After that, we grouped the dataset by the frequency of each venue category so we recognize what the most common venues are. Finally we used k-Means to cluster the zones that have high demand thus we know where exactly COVID-19 are increasing.